In [1]:
%ls data

coupon150720.csv  sales_segments.csv.gz*  transm150720.csv


In [3]:
# Se lee el archivo
data = sc.textFile("data/sales_segments.csv.gz")

In [4]:
# Se comprueba la lectura correcta
data.first()

u'rloc^departure_date_seg^booking_date^freq_flyer_flag^creator_office_id^creator_iata_number^creator_gds_code^creator_city_code^creator_lat^creator_lon^creator_country_code^creator_continent_code^creator_channel_code^booking_status_code^cabin_code^booking_class_code^pos_office_id^pos_iata_number^pos_gds_code^pos_city_code^pos_lat^pos_lon^pos_country_code^pos_continent_code^pos_channel_code^board_point_seg^board_lat^board_lon^board_country_code^board_continent_code^off_point_seg^off_lat^off_lon^off_country_code^off_continent_code^num_legs^distance_seg^route^network^airline_code^flight_number^flight_type^revenue_source^estimated_flag^usage_board_point_seg^usage_off_point_seg^usage_flight_number^usage_date^non_currency_type^bookings_seg^rpk_seg^revenue_amount_seg^currency_code^fuel_surcharge_amount_seg^fuel_surcharge_currency_code^emd_count^emd_amount^emd_currency_code^total_outliers'

In [5]:
header = data.first()

In [6]:
# Se elimina la cabecera del conjunto de datos
lines = data.filter(lambda l: l != header)

In [11]:
# Nos quedamos con los campos que nos interesan
fields = ['bookings_seg','revenue_amount_seg','fuel_surcharge_amount_seg']
# Se calculan las posiciones de cada campo
header.split('^').index('bookings_seg')

49

In [12]:
[header.split('^').index(field) for field in fields]

[49, 51, 53]

In [13]:
def splitData (line):
    elements = line.split("^")
    bookings = float(elements[49])
    rev = float(elements[51])
    tax = float(elements[53])
    
    av_rev = rev/bookings
    av_tax = tax/bookings
    
    return (av_rev, av_tax)

splitData(lines.first())

(33.56, 7.3)

In [14]:
simple = lines.map(splitData)

In [15]:
simple.take(3)

[(33.56, 7.3), (61.2, 25.5), (71.86842105263158, 12.31578947368421)]

In [16]:
simple.cache()

PythonRDD[6] at RDD at PythonRDD.scala:43

In [18]:
from pyspark.mllib.clustering import KMeans, KMeansModel
from numpy import array
from math import sqrt

In [19]:
# Entrenamiento del cluster
clusters = KMeans.train(simple, 10)

In [20]:
clusters.save(sc, "clusters")

In [23]:
clusters.centers

[array([ 117.58639715,   69.0546566 ]),
 array([  8.77500428e+05,   3.03020000e+01]),
 array([  3.72063199e+05,   1.03505455e+02]),
 array([  1.09277838e+05,   2.10108791e+01]),
 array([  1.39215574e+06,   3.27400000e+01]),
 array([  6.15206951e+05,   2.97312500e+01]),
 array([  4.77354917e+04,   3.31273780e+01]),
 array([  2.62467806e+05,   7.43850000e+01]),
 array([ 468.82400889,  189.60768202]),
 array([ 1640.820864  ,   208.04517166])]

In [24]:
clusters.clusterCenters

[array([ 117.58639715,   69.0546566 ]),
 array([  8.77500428e+05,   3.03020000e+01]),
 array([  3.72063199e+05,   1.03505455e+02]),
 array([  1.09277838e+05,   2.10108791e+01]),
 array([  1.39215574e+06,   3.27400000e+01]),
 array([  6.15206951e+05,   2.97312500e+01]),
 array([  4.77354917e+04,   3.31273780e+01]),
 array([  2.62467806e+05,   7.43850000e+01]),
 array([ 468.82400889,  189.60768202]),
 array([ 1640.820864  ,   208.04517166])]

In [26]:
# Dado un punto nos devuelve el cluster correspondiente
clusters.predict((33.56, 7.3))

0

In [27]:
# Se asigna el cluster correspondiente a cada punto
bycluster = simple.map(lambda point: (clusters.predict(point),point))

In [28]:
bycluster.take(3)

[(0, (33.56, 7.3)),
 (0, (61.2, 25.5)),
 (0, (71.86842105263158, 12.31578947368421))]

In [ ]:
# Queremos ejecutar lo siguiente, pero no lo podemos hacer así porque terminaría con la memoria
#bycluster.groupByKey().mapValues(lambda points: mean(points), stdev(points))

In [32]:
# Se meten los 1's en cada punto
bycluster_w = simple.map(lambda (x,y): (clusters.predict((x,y)),(x,y,1)))
bycluster_w.cache()
bycluster_w.take(3)

[(0, (33.56, 7.3, 1)),
 (0, (61.2, 25.5, 1)),
 (0, (71.86842105263158, 12.31578947368421, 1))]

In [33]:
def calc_avg_reduce(point, acum):
#    (rev_p, tax_p, c) = point
#    (rev_a, tax_a, a) = acum
#    result = (rev_p + rev_a, tax_p+tax_a, c+a)
#    return result

    point_x = point[0] + acum[0]
    point_y = point[1] + acum[1]
    point_c = point[2] + acum[2]
    
    return (point_x, point_y, point_c)

In [34]:
means = bycluster_w.reduceByKey(calc_avg_reduce)

In [35]:
means.take(3)

[(0, (134693571.72901854, 79101142.3676785, 1145486)),
 (1, (8775004.28, 303.02, 10)),
 (2, (4092695.19, 1138.5600000000002, 11))]

In [36]:
def medias(cluster):
    (rev_t, tax_t, count_t) = cluster
    return(rev_t/count_t, tax_t/count_t, count_t)

means_f = means.mapValues(medias)

In [37]:
means_f.take(3)

[(0, (117.5863971528404, 69.05465659787941, 1145486)),
 (1, (877500.428, 30.302, 10)),
 (2, (372063.1990909091, 103.50545454545455, 11))]

In [41]:
simple_j = bycluster.join(means_f)
simple_j.cache()

PythonRDD[113] at RDD at PythonRDD.scala:43

In [42]:
simple_j.take(3)

[(0, ((33.56, 7.3), (117.5863971528404, 69.05465659787941, 1145486))),
 (0, ((61.2, 25.5), (117.5863971528404, 69.05465659787941, 1145486))),
 (0,
  ((71.86842105263158, 12.31578947368421),
   (117.5863971528404, 69.05465659787941, 1145486)))]

In [43]:
def sq_diffs(point):
    ((x,y),(avg_x,avg_y,count)) = point
    return ((x-avg_x)**2, (y-avg_y)**2, 1)
# vamos a reutilizar la funcion calc_avg_reduce

In [44]:
points_w = simple_j.mapValues(sq_diffs)
points_w.take(3)

[(0, (7060.435418486866, 3813.637611522012, 1)),
 (0, (3179.425783877848, 1897.008111359201, 1)),
 (0, (2090.1333386992656, 3219.299042537079, 1))]

In [45]:
points_w_2 = points_w.reduceByKey(calc_avg_reduce)

In [46]:
points_w_2.take(3)

[(0, (7383943512.42626, 6298520271.110964, 1145486)),
 (8, (7977080974.873363, 940921069.1924549, 295462)),
 (2, (16806432286.595692, 165207.2056727273, 11))]

In [48]:
def stdevs(cluster):
    (sq_rev, sq_tax, n) = cluster
    
    std_rev = sqrt(sq_rev/n)
    std_tax = sqrt(sq_tax/n)
    
    return (std_rev, std_tax)

In [49]:
cluster_std = points_w_2.mapValues(stdevs)

In [50]:
cluster_std.take(3)

[(0, (80.28775049296158, 74.15225674965224)),
 (8, (164.3127195157817, 56.432043886483974)),
 (2, (39087.8175471541, 122.55136424919489))]

In [51]:
means_f.take(3)

[(0, (117.5863971528404, 69.05465659787941, 1145486)),
 (1, (877500.428, 30.302, 10)),
 (2, (372063.1990909091, 103.50545454545455, 11))]

In [52]:
cluster_stats = means_f.join(cluster_std)

In [53]:
cluster_stats.take(10)

[(0,
  ((117.5863971528404, 69.05465659787941, 1145486),
   (80.28775049296158, 74.15225674965224))),
 (9,
  ((1640.8208640001437, 208.0451716576782, 18003),
   (976.8069895588658, 63.4185824541965))),
 (3,
  ((109277.83769230757, 21.010879120879125, 91),
   (11744.553053653653, 66.54239499067711))),
 (6,
  ((47735.491707317095, 33.12737804878048, 41),
   (11349.61955513636, 36.59491219364705))),
 (1, ((877500.428, 30.302, 10), (47658.199670890375, 1.7160000000000006))),
 (4, ((1392155.74, 32.74, 1), (0.0, 0.0))),
 (7, ((262467.80625, 74.385, 8), (39946.08169664359, 66.4795444102921))),
 (8,
  ((468.8240088906417, 189.6076820214196, 295462),
   (164.3127195157817, 56.432043886483974))),
 (2,
  ((372063.1990909091, 103.50545454545455, 11),
   (39087.8175471541, 122.55136424919489))),
 (5,
  ((615206.9512499999, 29.731249999999996, 8),
   (49736.26150524646, 1.442501624782446)))]

In [54]:
bycluster.take(3)

[(0, (33.56, 7.3)),
 (0, (61.2, 25.5)),
 (0, (71.86842105263158, 12.31578947368421))]

In [56]:
bycluster_2 = bycluster.join(cluster_stats)
bycluster_2.cache()

PythonRDD[143] at RDD at PythonRDD.scala:43

In [57]:
bycluster_2.take(3)

[(0,
  ((33.56, 7.3),
   ((117.5863971528404, 69.05465659787941, 1145486),
    (80.28775049296158, 74.15225674965224)))),
 (0,
  ((61.2, 25.5),
   ((117.5863971528404, 69.05465659787941, 1145486),
    (80.28775049296158, 74.15225674965224)))),
 (0,
  ((71.86842105263158, 12.31578947368421),
   ((117.5863971528404, 69.05465659787941, 1145486),
    (80.28775049296158, 74.15225674965224))))]

In [74]:
def get_outlier(point):
    
    (p, stats) = point
    rev, tax = p
    means, stdev = stats
    (rev_avg, tax_avg, count) = means
    (rev_std, tax_std) = stdev
    
#    (rev,tax),((rev_avg,tax_avg,count),(rev_std,tax_std)) = point
    if rev_std == 0: 
        rev_std = 0.0001
    if tax_std == 0:
        tax_std = 0.0001
        
    z_rev = (rev-rev_avg)/rev_std
    z_tax = (tax-tax_avg)/tax_std

    # se calcula la hipotenusa
    z = sqrt(z_rev**2 + z_tax**2)
    
    return (rev, tax, z, z>3)

In [75]:
get_outlier(bycluster_2.first()[1])

(33.56, 7.3, 1.3374865010892811, False)

In [76]:
outlier = bycluster_2.mapValues(get_outlier)

In [77]:
outlier.take(3)

[(0, (33.56, 7.3, 1.3374865010892811, False)),
 (0, (61.2, 25.5, 0.9155499894001947, False)),
 (0, (71.86842105263158, 12.31578947368421, 0.9537962437272972, False))]

In [80]:
outlier.saveAsTextFile("data/outlier_spark")